In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd

In [2]:
data_dir = '/home/odin/chengbixiao/git_code/User_loan_demand/user_loan_demand/offline/data/'
train_info_path = data_dir + '0909/feature_model_file/train_info.csv'
test1_info_path = data_dir + '0909/feature_model_file/test1_info.csv'
test_info_path = data_dir + '0909/feature_model_file/test_info.csv'

fm_feature_train_norm_path = data_dir + '0909/feature_model_file/fm_feature_train.npy'
fm_feature_test1_norm_path = data_dir + '0909/feature_model_file/fm_feature_test1.npy'
fm_feature_test_norm_path = data_dir + '0909/feature_model_file/fm_feature_test.npy'

fm_data_path = data_dir + 'feature_ori_file/dsd_user_loan_demand_loan_repay_FM_feature_offline.csv'

fm_user_fins_feature_train_norm_path = data_dir + '0909/feature_model_file/fm_user_fins_feature_train.npy'
fm_user_fins_feature_test1_norm_path = data_dir + '0909/feature_model_file/fm_user_fins_feature_test1.npy'
fm_user_fins_feature_test_norm_path = data_dir + '0909/feature_model_file/fm_user_fins_feature_test.npy'
fm_user_people_feature_train_norm_path = data_dir + '0909/feature_model_file/fm_user_people_feature_train.npy'
fm_user_people_feature_test1_norm_path = data_dir + '0909/feature_model_file/fm_user_people_feature_test1.npy'
fm_user_people_feature_test_norm_path = data_dir + '0909/feature_model_file/fm_user_people_feature_test.npy'
fm_account_feature_train_norm_path = data_dir + '0909/feature_model_file/fm_account_feature_train.npy'
fm_account_feature_test1_norm_path = data_dir + '0909/feature_model_file/fm_account_feature_test1.npy'
fm_account_feature_test_norm_path = data_dir + '0909/feature_model_file/fm_account_feature_test.npy'


In [3]:
fm_data = pd.read_csv(fm_data_path,sep='\t')

In [4]:
fm_data.columns = [x.split('.')[1] for x in fm_data.columns]

In [5]:
fea_cols = fm_data.columns[5:]

In [6]:
fm_data[fea_cols] = fm_data[fea_cols].fillna(0)

In [7]:
fm_data.head()

,uid,funds_channel_id,tel,label,server_date,credit_line_level_1,credit_line_level_2,credit_line_level_3,credit_line_level_4,credit_line_level_5,...,company_occupation_1,company_occupation_2,company_occupation_3,company_occupation_4,company_occupation_5,company_occupation_6,company_occupation_7,company_occupation_8,company_occupation_9,is_businessman
0,72994,3038,18166584910,1,2020-08-05,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,72994,3038,18166584910,0,2020-08-07,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,72994,3038,18166584910,0,2020-08-06,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,72994,3038,18166584910,0,2020-08-09,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3669592,3038,13802429600,0,2020-08-05,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
def onhot(threshold1, x):
    if int(x) < 0.5:
        return 0
    elif int(x) >= 0.5:
        return 1
fm_data['is_businessman'] = fm_data['is_businessman'].apply(lambda x:onhot(0.5,x))

In [9]:
fm_data['is_businessman'].value_counts()

0    3514589
1      66995
Name: is_businessman, dtype: int64

In [10]:
train_info = pd.read_csv(train_info_path)
test1_info = pd.read_csv(test1_info_path)
test_info = pd.read_csv(test_info_path)

In [11]:
train_info.shape,test1_info.shape,test_info.shape

((812883, 4), (90320, 4), (96797, 4))

In [12]:
seq_info = pd.concat([train_info,test1_info,test_info],axis=0)
seq_info = seq_info.reset_index(drop=True)
train_index = seq_info.index.to_list()[:train_info.shape[0]]
test1_index = seq_info.index.to_list()[train_info.shape[0]:train_info.shape[0]+test1_info.shape[0]]
test_index = seq_info.index.to_list()[train_info.shape[0]+test1_info.shape[0]:]

In [13]:
len(train_index),len(test1_index),len(test_index)

(812883, 90320, 96797)

In [14]:
fm_data = fm_data.drop_duplicates(subset=['uid', 'server_date'], keep='first' )

In [15]:
fm_data.shape

(3580248, 112)

In [16]:
del fm_data['label']

In [17]:
fm_feature = pd.merge(seq_info, fm_data, on=['uid', 'server_date'],how='left')
print("GET TRAIN AND TEST INDEX FINISH!")

GET TRAIN AND TEST INDEX FINISH!


In [18]:
seq_info.head()

,Unnamed: 0,uid,server_date,label
0,898222,282455617110016,2020-08-03,0
1,650466,580542639297822,2020-07-27,0
2,358339,282651709014016,2020-07-17,0
3,359217,282713642369026,2020-07-23,0
4,819911,284376788505052,2020-07-14,0


In [19]:
fm_feature.head()

,Unnamed: 0,uid,server_date,label,funds_channel_id,tel,credit_line_level_1,credit_line_level_2,credit_line_level_3,credit_line_level_4,...,company_occupation_1,company_occupation_2,company_occupation_3,company_occupation_4,company_occupation_5,company_occupation_6,company_occupation_7,company_occupation_8,company_occupation_9,is_businessman
0,898222,282455617110016,2020-08-03,0,3024,13660786270,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,650466,580542639297822,2020-07-27,0,3018,18457901353,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,358339,282651709014016,2020-07-17,0,3015,13620085950,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
3,359217,282713642369026,2020-07-23,0,3024,18611783018,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,819911,284376788505052,2020-07-14,0,3021,15269760051,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [24]:
for x in fm_feature.columns[6:]:
    print(x)

credit_line_level_1
credit_line_level_2
credit_line_level_3
credit_line_level_4
credit_line_level_5
credit_line_level_6
current_drip_interest_rate_level_1
current_drip_interest_rate_level_2
current_drip_interest_rate_level_3
current_drip_interest_rate_level_4
current_drip_interest_rate_level_5
current_drip_interest_rate_level_6
current_drip_interest_rate_level_7
current_drip_interest_rate_level_8
current_drip_credit_line_balance_level_1
current_drip_credit_line_balance_level_2
current_drip_credit_line_balance_level_3
current_drip_credit_line_balance_level_4
current_drip_credit_line_balance_level_5
current_drip_credit_line_balance_level_6
plan_amount_level_1
plan_amount_level_2
plan_amount_level_3
plan_amount_level_4
plan_amount_level_5
plan_amount_level_6
plan_amount_level_7
plan_amount_level_8
plan_amount_level_9
duration_num_level_1
duration_num_level_2
duration_num_level_3
duration_num_level_4
duration_num_level_5
duration_num_level_6
current_valid_coupon_num_level_1
current_valid_c

In [20]:
sub_cols_user_fins = [
'last_credit_success_start_time_level_1',
'last_credit_success_start_time_level_2',
'last_credit_success_start_time_level_3',
'last_credit_success_start_time_level_4',
'last_credit_success_start_time_level_5',
'last_credit_success_end_time_level_6',
'ratio_6_level_1',
'ratio_6_level_2',
'ratio_6_level_3',
'ratio_6_level_4',
'ratio_6_level_5',
'last_6_month_drip_loan_num_level_1',
'last_6_month_drip_loan_num_level_2',
'last_6_month_drip_loan_num_level_3',
'last_6_month_drip_loan_num_level_4',
'last_6_month_drip_loan_num_level_5',
'last_6_month_drip_loan_money_level_1',
'last_6_month_drip_loan_money_level_2',
'last_6_month_drip_loan_money_level_3',
'last_6_month_drip_loan_money_level_4',
'last_6_month_drip_loan_money_level_5',
'last_6_month_drip_loan_money_level_6']

In [21]:
fm_user_fins_feature_train = fm_feature.loc[train_index,sub_cols_user_fins]
fm_user_fins_feature_test1 = fm_feature.loc[test1_index,sub_cols_user_fins]
fm_user_fins_feature_test = fm_feature.loc[test_index,sub_cols_user_fins]

In [22]:
sub_cols_user_people = ['fin_age_level_1',
'fin_age_level_2',
'fin_age_level_3',
'fin_age_level_4',
'fin_age_level_5',
'fin_age_level_6',
'fin_age_level_7',
'fin_age_level_8',
'is_beatles_driver',
'coin_level_1',
'coin_level_2',
'coin_level_3',
'coin_level_4',
'coin_level_5',
'coin_level_6',
'coin_level_7',
'coin_level_8',
'first_call_days_level_1',
'first_call_days_level_2',
'first_call_days_level_3',
'first_call_days_level_4',
'first_call_days_level_5',
'first_call_days_level_6',
'first_call_days_level_7',
'hit_loan_app_cnt_level_1',
'hit_loan_app_cnt_level_2',
'hit_loan_app_cnt_level_3',
'hit_loan_app_cnt_level_4',
'hit_loan_app_cnt_level_5',
'price_rank_level_1',
'price_rank_level_2',
'price_rank_level_3',
'price_rank_level_4',
'price_rank_level_5',
'pas_sex',
'is_fast_or_gulf_driver',
'company_occupation_1',
'company_occupation_2',
'company_occupation_3',
'company_occupation_4',
'company_occupation_5',
'company_occupation_6',
'company_occupation_7',
'company_occupation_8',
'company_occupation_9',
'is_businessman']

In [23]:
fm_user_people_feature_train = fm_feature.loc[train_index,sub_cols_user_people]
fm_user_people_feature_test1 = fm_feature.loc[test1_index,sub_cols_user_people]
fm_user_people_feature_test = fm_feature.loc[test_index,sub_cols_user_people]

In [24]:
# 5.split into train and test 
fm_account_feature_train = fm_feature.iloc[train_index,  6:6+39]
fm_account_feature_test1 = fm_feature.iloc[test1_index,  6:6+39]
fm_account_feature_test = fm_feature.iloc[test_index,  6:6+39]
print("SPLIT INTO TRAIN AND TEST FINISH !")



SPLIT INTO TRAIN AND TEST FINISH !


In [25]:
fm_user_fins_feature_train.shape,fm_user_fins_feature_test1.shape,fm_user_fins_feature_test.shape,

((812883, 22), (90320, 22), (96797, 22))

In [26]:
fm_user_people_feature_train.shape,fm_user_people_feature_test1.shape,fm_user_people_feature_test.shape

((812883, 46), (90320, 46), (96797, 46))

In [27]:
fm_account_feature_train.shape,fm_account_feature_test1.shape,fm_account_feature_test.shape

((812883, 39), (90320, 39), (96797, 39))

In [35]:
22+46+39

107

In [28]:
fm_feature_train = fm_feature.iloc[train_index,  6:]
fm_feature_test1 = fm_feature.iloc[test1_index,  6:]
fm_feature_test = fm_feature.iloc[test_index,  6:]
print("SPLIT INTO TRAIN AND TEST FINISH !")
#  6.save
np.save(fm_feature_train_norm_path,fm_feature_train)
np.save(fm_feature_test1_norm_path,fm_feature_test1)
np.save(fm_feature_test_norm_path,fm_feature_test)
print("SAVE FINISH !")

SPLIT INTO TRAIN AND TEST FINISH !
SAVE FINISH !


In [30]:
# fm_feature_train.head()

In [31]:
np.save(fm_user_fins_feature_train_norm_path,fm_user_fins_feature_train)
np.save(fm_user_fins_feature_test1_norm_path,fm_user_fins_feature_test1)
np.save(fm_user_fins_feature_test_norm_path,fm_user_fins_feature_test)
print("SAVE FINISH !")
np.save(fm_user_people_feature_train_norm_path,fm_user_people_feature_train)
np.save(fm_user_people_feature_test1_norm_path,fm_user_people_feature_test1)
np.save(fm_user_people_feature_test_norm_path,fm_user_people_feature_test)
print("SAVE FINISH !")
np.save(fm_account_feature_train_norm_path,fm_account_feature_train)
np.save(fm_account_feature_test1_norm_path,fm_account_feature_test1)
np.save(fm_account_feature_test_norm_path,fm_account_feature_test)
print("SAVE FINISH !")

SAVE FINISH !
SAVE FINISH !
SAVE FINISH !
